# Unsupervised

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

from sklearn.cluster import KMeans
import nltk
from sklearn.cluster import AffinityPropagation
from sklearn.cluster import KMeans

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE

## Kmeans Model

### Data Preprocessing1(using leaves) for Kmeans Model

In [2]:
stopwords_dict = os.listdir("stopwords/")
stop_list = []
for dic in stopwords_dict:
    stop_list.append([line.strip() for line in open('stopwords/' + dic, 'r').readlines()])
with open('level0_dataset1_cleaned_cle_rep.csv') as f:
    raw_word = pd.read_csv('level0_dataset1_cleaned_cle_rep.csv') #,encoding = 'gbk'
with open('sentence_dataset1.csv') as f:
    raw_sentence = pd.read_csv('sentence_dataset1.csv') #,encoding = 'gbk'
    
dic_len = len(raw_word)
sen_len = len(raw_sentence)

dic = []
dic = [i for i in raw_word['phrase']]

dictionary = pd.DataFrame({"phrase":dic})
dictionary.to_csv('dictionary.csv',index = False)

words = []
for i in raw_sentence['phrase']:
    word = []
    word = i.split(' ')
    clean = []
    for j in word:
        if j not in stop_list and j not in clean:
            clean.append(j)
    words.append(clean)

### One-hot

In [3]:
code = np.zeros((sen_len,dic_len),dtype='int')
for j in range(sen_len):
    # j = 0
    for t in words[j]:
        if t in dic:
            index = dic.index(t)
            code[j][index] = 1
#     print(j,':',code[j])

n = 15 # we can modified it
julei=KMeans(n_clusters=n)
julei.fit(code)

label=julei.labels_
center=julei.cluster_centers_ 

label = np.array(label)

sentimental =  list(raw_sentence['label'])
mean = []
var = []
length = []
for i in range(n):
    test = []
    for t in range(len(label)):
        if i == label[t]:           
            test.append(sentimental[t])
    test = np.array(test,dtype = 'float')
    mean.append(round(np.mean(test),2))
    var.append(round(np.var(test),4))
    length.append(len(test))
    
mv = pd.DataFrame({'mean':mean,'var':var,'len':length})
mv[mv['len']>10]


,mean,var,len
1,0.52,0.0622,4949
2,0.47,0.0730,636
4,0.49,0.0616,1081
5,0.61,0.0948,12
6,0.53,0.0585,309
7,0.51,0.0598,217
8,0.55,0.0616,713
10,0.49,0.0637,619



### Encoding by sentiment labels(using leaves)


In [4]:
# get dictionary
level = []
for i in range(len(raw_word)):
    if raw_word['label'][i] <= 0.2:
        level.append(0)
    elif raw_word['label'][i] <= 0.4:
        level.append(1)
    elif raw_word['label'][i] <= 0.6:
        level.append(2)
    elif raw_word['label'][i] <= 0.8:
        level.append(3)
    else:
        level.append(4)
        
dic_len = len(raw_word)

dic = []
dic = [i for i in raw_word['phrase']]

dictionary = pd.DataFrame({"phrase":dic,'level':level})
dictionary.to_csv('dictionary_level.csv',index = False)

code = np.zeros((sen_len,5),dtype='int')
for j in range(sen_len):
    for t in words[j]:
        if t in dic:
            index = dic.index(t)
            code[j][level[index]] += 1
        for i in range(5):
            if sum(code[j]) > 0:
                code[j][i] = 20*(code[j][i]/sum(code[j]))

n = 100
julei=KMeans(n_clusters=n)
julei.fit(code)

label=julei.labels_
center=julei.cluster_centers_ 

label = np.array(label)

sentimental =  list(raw_sentence['label'])
mean = []
var = []
length = []
for i in range(n):
    test = []
    for t in range(len(label)):
        if i == label[t]:           
            test.append(sentimental[t])
    test = np.array(test,dtype = 'float')
    mean.append(round(np.mean(test),2))
    var.append(round(np.var(test),4))
    length.append(len(test))
    
mv = pd.DataFrame({'mean':mean,'var':var,'len':length})
mv[mv['len']>100]


,mean,var,len
0,0.62,0.0561,487
1,0.54,0.0585,381
3,0.40,0.0480,2431
7,0.48,0.0573,1993
8,0.54,0.0599,960
11,0.59,0.0595,377
12,0.59,0.0592,517
17,0.63,0.0496,276
21,0.67,0.0502,108
24,0.68,0.0560,130


### Encoding by sentiment labels(using all nodes)

In [5]:
stopwords_dict = os.listdir("stopwords/")
stop_list = []
for dic in stopwords_dict:
    stop_list.append([line.strip() for line in open('stopwords/' + dic, 'r').readlines()])
stop_words = ' '.join(str(i) for i in stop_list)


raw_word = pd.read_csv('level0_dataset1_cleaned_cle_rep.csv') #,encoding = 'gbk'

raw_data = pd.read_csv('Node_level_label.csv',encoding = 'gbk') #,encoding = 'gbk'
sen_len = max(raw_data['sentence_id'])

code = np.zeros((sen_len+1,5),dtype='int')
for j in range(sen_len+1):
    s_index = raw_data.loc[raw_data['sentence_id'] == j].index
    for i in range(5):
        for t in s_index:
            if raw_data['level'][t] == i:
                code[j][i] += 1
        for i in range(5):
            if sum(code[j]) > 0:
                code[j][i] = 20*(code[j][i]/sum(code[j]))

n = 20
julei=KMeans(n_clusters=n)
julei.fit(code)

label=julei.labels_
center=julei.cluster_centers_ 

label = np.array(label)
a = pd.read_csv('sentence_level.csv')
sentimental =  list(a['label'])
mean = []
var = []
length = []
for i in range(n):
    test = []
    for t in range(len(label)):
        if i == label[t]:           
            test.append(sentimental[t])
    test = np.array(test,dtype = 'float')
    mean.append(round(np.mean(test),2))
    var.append(round(np.var(test),4))
    length.append(len(test))
    
mv = pd.DataFrame({'mean':mean,'var':var,'len':length})
mv[mv['len']>100]

,mean,var,len
0,0.77,0.0178,477
1,0.40,0.0209,744
2,0.19,0.0209,208
3,0.65,0.0367,692
4,0.43,0.0356,301
5,0.71,0.0323,483
6,0.65,0.0449,821
7,0.42,0.0228,788
8,0.24,0.0218,578
9,0.83,0.0157,458


### Add height weight

In [6]:
stopwords_dict = os.listdir("stopwords/")
stop_list = []
for dic in stopwords_dict:
    stop_list.append([line.strip() for line in open('stopwords/' + dic, 'r').readlines()])
stop_words = ' '.join(str(i) for i in stop_list)


raw_word = pd.read_csv('level0_dataset1_cleaned_cle_rep.csv') #,encoding = 'gbk'

raw_data = pd.read_csv('Node_level_label_height.csv',encoding = 'gbk') #,encoding = 'gbk'
sen_len = max(raw_data['sentence_id'])

code = np.zeros((sen_len+1,5),dtype='int')
for j in range(sen_len+1):
    s_index = raw_data.loc[raw_data['sentence_id'] == j].index
    for i in range(5):
        for t in s_index:
            if raw_data['level'][t] == i:
#                 code[j][i] += 1
                code[j][i] += 1*raw_data['height_level'][t]
#                 if raw_data['height_level'][t]<=0.75:
#                     code[j][i] += 1
# #                     code[j][i] += 1*raw_data['height_level'][t]
        for i in range(5):
            if sum(code[j]) > 0:
                code[j][i] = 20*(code[j][i]/sum(code[j]))

n = 5
julei=KMeans(n_clusters=n)
julei.fit(code)

label=julei.labels_
center=julei.cluster_centers_ 

label = np.array(label)
sentence_level = pd.read_csv('sentence_level.csv')
sentimental =  list(sentence_level['label'])
mean = []
var = []
length = []
for i in range(n):
    test = []
    for t in range(len(label)):
        if i == label[t]:           
            test.append(sentimental[t])
    test = np.array(test,dtype = 'float')
    mean.append(round(np.mean(test),2))
    var.append(round(np.var(test),4))
    length.append(len(test))
    
mv = pd.DataFrame({'mean':mean,'var':var,'len':length})
mv[mv['len']>10]

,mean,var,len
0,0.29,0.0030,3140
1,0.71,0.0030,3111
2,0.86,0.0022,1852
3,0.50,0.0034,2242
4,0.13,0.0025,1510
